### Pyiceburg

* [pyiceburg documentation](https://py.iceberg.apache.org/#write-a-pyarrow-dataframe)

In [ ]:
%pip install -r requirements.txt

In [4]:
import os

import pyarrow.parquet as pq
import pyarrow.compute as pc
from pyiceberg.catalog.sql import SqlCatalog

In [5]:
warehouse_path = "/tmp/warehouse"
os.makedirs(warehouse_path, exist_ok=True)

In [6]:
catalog = SqlCatalog(
    "default",
    **{
        "uri": f"sqlite:///{warehouse_path}/pyiceberg_catalog.db",
        "warehouse": f"file://{warehouse_path}",
    },
)
namespace = "default"

In [ ]:
!curl https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet -o /tmp/yellow_tripdata_2023-01.parquet


In [ ]:
!ls /tmp/warehouse

In [ ]:
df = pq.read_table("/tmp/yellow_tripdata_2023-01.parquet")
df.schema

In [ ]:
catalog.create_namespace(namespace)
table = catalog.create_table(
    f"{namespace}.texi_dataset",
    schema=df.schema
)

In [7]:
table = catalog.load_table(f"{namespace}.texi_dataset")
table

texi_dataset(
  1: VendorID: optional long,
  2: tpep_pickup_datetime: optional timestamp,
  3: tpep_dropoff_datetime: optional timestamp,
  4: passenger_count: optional double,
  5: trip_distance: optional double,
  6: RatecodeID: optional double,
  7: store_and_fwd_flag: optional string,
  8: PULocationID: optional long,
  9: DOLocationID: optional long,
  10: payment_type: optional long,
  11: fare_amount: optional double,
  12: extra: optional double,
  13: mta_tax: optional double,
  14: tip_amount: optional double,
  15: tolls_amount: optional double,
  16: improvement_surcharge: optional double,
  17: total_amount: optional double,
  18: congestion_surcharge: optional double,
  19: airport_fee: optional double,
  20: tip_per_mile: optional double
),
partition by: [],
sort order: [],
snapshot: Operation.OVERWRITE: id=2658357625130709920, parent_id=5367457311721339848, schema_id=1

In [10]:
table.append(df)
len(table.scan().to_arrow())

3066766

In [ ]:
df = df.append_column("tip_per_mile", pc.divide(df["tip_amount"], df["trip_distance"]))

In [ ]:
with table.update_schema() as update_schema:
    update_schema.union_by_name(df.schema)

In [ ]:
table.overwrite(df)
print(table.scan().to_arrow())

In [ ]:
df = table.scan(row_filter="tip_per_mile > 0").to_arrow()
len(df)

In [ ]:
!find /tmp/warehouse/

In [ ]:
table

In [9]:
table.scan().to_pandas()

: 